# 🛡️ Solutions — Experimental Topics in Differential Privacy

Built by **Stu** 🚀

## Solutions to Exercises 1–6 (Tiny Synthetic Data + DP-GAN)

In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)
df = pd.DataFrame({
    'Age': np.random.randint(18, 70, size=500),
    'Salary': np.random.randint(20000, 200000, size=500),
    'PurchaseAmount': np.random.uniform(5, 1000, size=500),
    'IsTraveler': np.random.choice(['Yes', 'No'], size=500),
    'Country': np.random.choice(['USA', 'UK', 'FR', 'JP', 'IN'], size=500)
})
df.head()

In [2]:
def laplace_mechanism(value, sensitivity, epsilon):
    scale = sensitivity / epsilon
    return value + np.random.laplace(0, scale)

df_noisy = df.copy()
for col in ['Age', 'Salary', 'PurchaseAmount']:
    df_noisy[col] = df_noisy[col].apply(lambda x: laplace_mechanism(x, 1, 1.0))
df_noisy.head()

In [3]:
import matplotlib.pyplot as plt

plt.hist(df['Age'], alpha=0.5, label='Real')
plt.hist(df_noisy['Age'], alpha=0.5, label='Noisy')
plt.legend()
plt.title('Age Feature: Real vs Noisy')
plt.show()

In [4]:
import torch
import torch.nn as nn

class TinyGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 5)
    def forward(self, x):
        return torch.tanh(self.fc(x))

generator = TinyGenerator()

In [5]:
class TinyDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(5, 1)
    def forward(self, x):
        return torch.sigmoid(self.fc(x))

discriminator = TinyDiscriminator()

In [6]:
def add_dp_noise(output, sigma=1.0):
    return output + torch.normal(0, sigma, size=output.shape)

fake_output = discriminator(torch.randn(1,5))
dp_output = add_dp_noise(fake_output)
dp_output

## Solutions to Exercises 7–16 (Adaptive FL, Tiny LLM, Secure Aggregation, MIA)

In [7]:
client_epsilons = {f'client_{i}': np.random.uniform(0.5, 2.0) for i in range(10)}
client_epsilons

In [8]:
participation_probs = {client: 1/eps for client, eps in client_epsilons.items()}
participation_probs

In [9]:
tiny_text = ["buy", "book", "travel", "flight", "hotel", "room"]
tiny_text

In [10]:
class TinyTextModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 8)
        self.fc = nn.Linear(8, vocab_size)
    def forward(self, x):
        x = self.embedding(x)
        return self.fc(x)

model = TinyTextModel(len(tiny_text))

In [11]:
def dp_sgd_step(gradients, sigma):
    noisy_gradients = gradients + torch.normal(0, sigma, size=gradients.shape)
    return noisy_gradients

In [12]:
for param in model.parameters():
    if param.grad is not None:
        param.grad = dp_sgd_step(param.grad, sigma=1.0)

In [13]:
client_updates = [np.random.randn(5) for _ in range(10)]
secure_sum = np.sum(client_updates, axis=0)
secure_sum

In [14]:
def add_secure_dp_noise(vector, sigma):
    noise = np.random.normal(0, sigma, size=vector.shape)
    return vector + noise

secure_dp_sum = add_secure_dp_noise(secure_sum, 1.0)
secure_dp_sum

In [15]:
mia_success_rate = np.random.uniform(0.4, 0.6)
mia_success_rate

In [16]:
mia_defense_sketch = "Lower ε to increase noise, making membership inference harder. Trade-off: higher noise decreases model accuracy."